In [1]:
import pandas as pd
import re
import emoji
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE

In [2]:
normal_data = pd.read_csv(r"C:\Users\limti\PycharmProjects\DSA4263_StockTweets\data\normal_data.csv")
tfidf_data = pd.read_csv(r"C:\Users\limti\PycharmProjects\DSA4263_StockTweets\data\tfidf_data.csv")

In [3]:
def count_hashtags(text):
    if isinstance(text, str):
        return len(re.findall(r'#\w+', text))
    return 0

def count_emojis(text):
    if isinstance(text, str):
        return len([char for char in text if char in emoji.EMOJI_DATA])
    return 0


# Apply the functions to your text column
normal_data['hashtag_count'] = normal_data['text'].apply(count_hashtags) #replace text_column
normal_data['emoji_count'] = normal_data['text'].apply(count_emojis) #replace text_column

# Display the DataFrame with the new columns
print(normal_data[['text', 'hashtag_count', 'emoji_count']].head())

                                                text  hashtag_count  \
0                    #rt  rt    axtg new intern ceo               1   
1                rt  #rt  rt    axtg new intern ceo               1   
2             axtg big otc  gainerrocketrocketrocket              0   
3  todays top penny stock gainers\n\nlttgf axtg s...              1   
4  rt  todays top penny stock gainers\n\nlttgf ax...              3   

   emoji_count  
0            0  
1            0  
2            0  
3            0  
4            0  


In [4]:
data = normal_data[["text", "Known_Pumper"]]
print(data.head())

                                                text  Known_Pumper
0                    #rt  rt    axtg new intern ceo              0
1                rt  #rt  rt    axtg new intern ceo              0
2             axtg big otc  gainerrocketrocketrocket             0
3  todays top penny stock gainers\n\nlttgf axtg s...             0
4  rt  todays top penny stock gainers\n\nlttgf ax...             0


In [32]:
import pandas as pd
import re
import emoji
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE

def count_hashtags(text):
    if isinstance(text, str):
        return len(re.findall(r'#\w+', text))
    return 0

def count_emojis(text):
    if isinstance(text, str):
        return len([char for char in text if char in emoji.EMOJI_DATA])
    return 0

def preprocess_text(text):
    if isinstance(text, str):
        text = emoji.demojize(text)
        text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
        text = re.sub(r'@\w+', '', text)
        text = re.sub(r'[^\w\s#]', '', text)
        text = re.sub(r'[^a-zA-Z\s#]', '', text)
        text = text.lower()
        return text
    else:
        return ""

def align_tfidf_columns(train_tfidf_df, test_tfidf_df):
    """Aligns TF-IDF columns between training and test DataFrames."""
    train_cols = set(train_tfidf_df.columns)
    test_cols = set(test_tfidf_df.columns)

    common_cols = list(train_cols & test_cols)
    missing_train_cols = list(test_cols - train_cols)
    missing_test_cols = list(train_cols - test_cols)

    for col in missing_train_cols:
        train_tfidf_df[col] = 0

    for col in missing_test_cols:
        test_tfidf_df[col] = 0

    return train_tfidf_df[common_cols + missing_test_cols], test_tfidf_df[common_cols + missing_train_cols]

def create_ml_pipeline_with_tfidf_oversampling(df, classifier, text_column='text', target_column='Known_Pumper', numerical_columns=["emoji_count", "hashtag_count"], oversampling=True, max_tfidf_features=100, param_grid=None):
    """
    Creates a machine learning pipeline with TF-IDF, emoji/hashtag counts, and optional SMOTE oversampling.
    Returns metrics for both train and test sets.
    """

    # Feature Engineering (Emoji and Hashtag Counts)
    df['hashtag_count'] = df[text_column].apply(count_hashtags)
    df['emoji_count'] = df[text_column].apply(count_emojis)

    # Text Preprocessing for TF-IDF
    df[text_column] = df[text_column].apply(preprocess_text)

    # Separate features and target
    X = df.drop(target_column, axis=1)
    y = df[target_column]

    # Train-Test Split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # TF-IDF Vectorization
    vectorizer = TfidfVectorizer(max_features=max_tfidf_features, stop_words='english')
    train_tfidf_matrix = vectorizer.fit_transform(X_train[text_column])
    test_tfidf_matrix = vectorizer.transform(X_test[text_column])

    train_tfidf_df = pd.DataFrame(train_tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
    test_tfidf_df = pd.DataFrame(test_tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

    # Align TF-IDF Columns
    train_tfidf_df, test_tfidf_df = align_tfidf_columns(train_tfidf_df, test_tfidf_df)

    # Feature Concatenation
    X_train = X_train.reset_index(drop=True)
    train_tfidf_df = train_tfidf_df.reset_index(drop=True)
    X_test = X_test.reset_index(drop=True)
    test_tfidf_df = test_tfidf_df.reset_index(drop=True)
    
    X_train = pd.concat([X_train.drop(text_column, axis=1), train_tfidf_df], axis=1)
    X_test = pd.concat([X_test.drop(text_column, axis=1), test_tfidf_df], axis=1)

    # Oversampling using SMOTE (if enabled)
    if oversampling:
        smote = SMOTE(random_state=42)
        X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
    else:
        X_train_resampled, y_train_resampled = X_train, y_train

    # Identify TF-IDF columns
    tfidf_columns = [col for col in X_train.columns if col not in numerical_columns and col != 'hashtag_count' and col != 'emoji_count']

    # Preprocessing pipeline
    preprocessor = ColumnTransformer(
        transformers=[
            ('numerical', MinMaxScaler(), numerical_columns + ['hashtag_count', 'emoji_count']),
            ('tfidf', 'passthrough', tfidf_columns)
        ],
        remainder='drop'
    )

    # Full pipeline
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', classifier)
    ])

    # Hyperparameter tuning with GridSearchCV (if enabled)
    if param_grid is not None:
        grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='f1', verbose=2, n_jobs=3)
        grid_search.fit(X_train_resampled, y_train_resampled)
        pipeline = grid_search.best_estimator_

    # Train the pipeline
    else:
        pipeline.fit(X_train_resampled, y_train_resampled)

    # Make predictions
    y_train_pred = pipeline.predict(X_train_resampled)
    y_test_pred = pipeline.predict(X_test)

    # Calculate metrics
    train_metrics = {
        'accuracy': accuracy_score(y_train_resampled, y_train_pred),
        'precision': precision_score(y_train_resampled, y_train_pred),
        'recall': recall_score(y_train_resampled, y_train_pred),
        'f1': f1_score(y_train_resampled, y_train_pred),
        'roc_auc': roc_auc_score(y_train_resampled, pipeline.predict_proba(X_train_resampled)[:, 1]),
        'confusion_matrix': confusion_matrix(y_train_resampled, y_train_pred)
    }

    test_metrics = {
        'accuracy': accuracy_score(y_test, y_test_pred),
        'precision': precision_score(y_test, y_test_pred),
        'recall': recall_score(y_test, y_test_pred),
        'f1': f1_score(y_test, y_test_pred),
        'roc_auc': roc_auc_score(y_test, pipeline.predict_proba(X_test)[:, 1]),
        'confusion_matrix': confusion_matrix(y_test, y_test_pred)
    }
    
    return pipeline, vectorizer, train_metrics, test_metrics

In [37]:
from sklearn.linear_model import LogisticRegression

param_grid_logistic = {
    'classifier__penalty': ['l1', 'l2'],  # Regularization type
    'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100],  # Inverse of regularization strength
    'classifier__solver': ['liblinear', 'saga'],  # Solver algorithm
    'classifier__class_weight': [None, 'balanced'] #Handle Imbalance
}

pipeline, vectorizer, train_metrics, test_metrics = create_ml_pipeline_with_tfidf_oversampling(data, classifier = LogisticRegression(), param_grid = param_grid_logistic)

C:\Users\limti\AppData\Local\Temp\ipykernel_10932\187949829.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['hashtag_count'] = df[text_column].apply(count_hashtags)
C:\Users\limti\AppData\Local\Temp\ipykernel_10932\187949829.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['emoji_count'] = df[text_column].apply(count_emojis)
C:\Users\limti\AppData\Local\Temp\ipykernel_10932\187949829.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

Fitting 5 folds for each of 48 candidates, totalling 240 fits


In [29]:
train_metrics

{'accuracy': 0.814983922829582,
 'precision': 0.7935043144774688,
 'recall': 0.8515755627009647,
 'f1': 0.8215149823190024,
 'roc_auc': np.float64(0.8784621829799113),
 'confusion_matrix': array([[6052, 1723],
        [1154, 6621]])}

In [30]:
test_metrics

{'accuracy': 0.7738095238095238,
 'precision': 0.11359026369168357,
 'recall': 0.7466666666666667,
 'f1': 0.19718309859154928,
 'roc_auc': np.float64(0.8163111798042246),
 'confusion_matrix': array([[1504,  437],
        [  19,   56]])}

In [38]:
import os
import pickle
pipeline_path = os.path.join(r"C:\Users\limti\PycharmProjects\DSA4263_StockTweets\model_training\tfidf_training", "lr_pipeline.pkl")
vectorizer_path = os.path.join(r"C:\Users\limti\PycharmProjects\DSA4263_StockTweets\model_training\tfidf_training", "lr_vectorizer.pkl")

with open(pipeline_path, 'wb') as p:
    pickle.dump(pipeline, p)

with open(vectorizer_path, 'wb') as v:
    pickle.dump(vectorizer, v)

In [33]:
from sklearn.ensemble import GradientBoostingClassifier

param_grid_gradient_boosting = {
    'classifier__n_estimators': [100, 200],  # Reduced number of estimators
    'classifier__learning_rate': [0.05, 0.1],  # Reduced learning rate options
    'classifier__max_depth': [3, 4],  # Reduced depth options
    'classifier__subsample': [0.9, 1.0],  # Reduced subsample options
    'classifier__max_features': ['sqrt', None] # Reduced number of features to consider.
}


pipeline, vectorizer, train_metrics, test_metrics = create_ml_pipeline_with_tfidf_oversampling(data, classifier = GradientBoostingClassifier(), param_grid=param_grid_gradient_boosting)

C:\Users\limti\AppData\Local\Temp\ipykernel_10932\187949829.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['hashtag_count'] = df[text_column].apply(count_hashtags)
C:\Users\limti\AppData\Local\Temp\ipykernel_10932\187949829.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['emoji_count'] = df[text_column].apply(count_emojis)
C:\Users\limti\AppData\Local\Temp\ipykernel_10932\187949829.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

Fitting 5 folds for each of 32 candidates, totalling 160 fits


In [34]:
train_metrics

{'accuracy': 0.9008360128617363,
 'precision': 0.8823457244509876,
 'recall': 0.925016077170418,
 'f1': 0.9031771945246766,
 'roc_auc': np.float64(0.9640159138139598),
 'confusion_matrix': array([[6816,  959],
        [ 583, 7192]])}

In [35]:
test_metrics

{'accuracy': 0.8472222222222222,
 'precision': 0.13479623824451412,
 'recall': 0.5733333333333334,
 'f1': 0.2182741116751269,
 'roc_auc': np.float64(0.8024283015627685),
 'confusion_matrix': array([[1665,  276],
        [  32,   43]])}

In [36]:
import os
import pickle
pipeline_path = os.path.join(r"C:\Users\limti\PycharmProjects\DSA4263_StockTweets\model_training\tfidf_training", "gb_pipeline.pkl")
vectorizer_path = os.path.join(r"C:\Users\limti\PycharmProjects\DSA4263_StockTweets\model_training\tfidf_training", "gb_vectorizer.pkl")

with open(pipeline_path, 'wb') as p:
    pickle.dump(pipeline, p)

with open(vectorizer_path, 'wb') as v:
    pickle.dump(vectorizer, v)

In [40]:
data["Known_Pumper"]

0        0
1        0
2        0
3        0
4        0
        ..
10071    0
10072    0
10073    0
10074    0
10075    0
Name: Known_Pumper, Length: 10076, dtype: int64